In [17]:
import numpy as np
import pandas as pd
import requests
import camelot

In [ ]:
#pip uninstall camelot -y
#pip uninstall camelot-py -y
#pip install camelot-py[all]

In [ ]:
r = requests.get("https://www.ard-zdf-massenkommunikation.de/files/Download-Archiv/MK_2020/MK_Langzeitstudie_2020_-_Mediennutzung_im_OstWest-Vergleich.pdf")

In [ ]:
open('data.pdf', 'wb').write(r.content)
import textract
text = textract.process('data.pdf').decode("utf-8")

In [2]:
tables = camelot.read_pdf('data.pdf', flavor='stream', split_text=True, pages='all')

In [3]:
def get_tab_l(inp):
    an=[]
    en=[]
    for i in inp.df.iloc:
        if any(i.str.contains("Tabelle"))==True:
            an.append(i.name)
        if any(i.str.contains("Quelle"))==True:
            en.append(int(i.name)+1)
        else:
            pass
    return list(zip(np.asarray(an),np.asarray(en)))
            
            
ll = []           
for i in tables:
    ll.append(get_tab_l(i))

In [4]:
tab_list = []
for i in range(len(ll)):
    for j in ll[i]:
        try:
            zw = tables[i].df[j[0]:j[1]]
            if len(zw)>0:
                tab_list.append(zw.reset_index(drop=True))
            else:
                pass
        except:
            pass
del tab_list[5]

In [301]:
def get_tab_abl(inp):
    an=[]
    en=[]
    for i in inp.iloc:
        if any(i.str.contains("ABL"))==True:
            if any(i.str.contains("ABL="))==True:
                en.append(i.name)
            else:
                an.append(i.name)
        else:
            pass
    return list(zip(np.asarray(an),np.asarray(en)))

tabel_dic={}
ldl=[]
for i in tab_list:
    nn=" ".join(["".join(s) for s in i.loc[:2].values])
    anen=get_tab_abl(i)
    tt=i.loc[anen[0][0]+1:anen[0][1]-1]
    tto=tt.set_index(0)
    tto=tto.replace("",np.nan).dropna()
    tto.index.name=nn
    try:
        tto.columns=["Alte Bundesländer","Neue Bundesländer","Differenz"]
        tto["Alte Bundesländer"] = pd.to_numeric(tto["Alte Bundesländer"])
        tto["Neue Bundesländer"] = pd.to_numeric(tto["Neue Bundesländer"])
        tto["Differenz"] = tto["Neue Bundesländer"]-tto["Alte Bundesländer"]
        test=tto
        tabel_dic.update({nn : tto})
        ldl.append(nn)
    except:
        pass

In [309]:
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import plotly.graph_objects as go

def f(Tabelle):
    import plotly.graph_objects as go
    df = tabel_dic[Tabelle]
    fig = go.Figure(data=[
    go.Bar(name="Alte Bundesländer", x=df.index, y=df["Alte Bundesländer"],marker_color='#FA7D19'),
    go.Bar(name="Neue Bundesländer", x=df.index, y=df["Neue Bundesländer"],marker_color='#999999'),
    go.Bar(name="Differenz", x=df.index, y=df["Differenz"],text=df["Differenz"],marker_color='#666666')
    ])
    fig.update_traces(textposition='outside')
    fig.update_layout(barmode='group', title=df.index.name,title_font_family='Swiss 721 BT')
    date_=Tabelle[:10]
    fig.show()
    return Tabelle
interact(f, Tabelle=ldl);

interactive(children=(Dropdown(description='Tabelle', options=('Tabelle 1 Demografie und Berufstätigkeit in Os…

In [286]:
buttons_list=[]
for i in ldl:
    zw=dict(args=['shapes', tabel_dic[i]],label=i[:10],method="update")
    buttons_list.append(zw)

In [288]:
import plotly.graph_objects as go

df = tabel_dic[ldl[0]]

fig = go.Figure(data=[
    go.Bar(name="Alte Bundesländer", x=df.index, y=df["Alte Bundesländer"],marker_color='#FA7D19'),
    go.Bar(name="Neue Bundesländer", x=df.index, y=df["Neue Bundesländer"],marker_color='#999999'),
    go.Bar(name="Differenz", x=df.index, y=df["Differenz"],text=df["Differenz"],marker_color='#666666')
])
fig.update_traces(textposition='outside')
fig.update_layout(barmode='group', title=df.index.name,title_font_family='Swiss 721 BT')

fig.update_layout(
    updatemenus=[
        dict(
            buttons=buttons_list,
            direction="down",
            pad={"r": 10, "t": 10},
            showactive=True,
            x=-0.1,
            xanchor="left",
            y=1.17,
            yanchor="top"
        ),
    ]
)

fig.show()